In [43]:
import pandas as pd
import io

In [94]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

i = 1
df_24, _ = _process_data_(f'2024_data_logs/round_{i}.log')
df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

j = 2
df_23 = pd.read_csv(f"2023_data_logs/r{j}.csv", sep=';')

df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]

df_24.columns  = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]

df_test = df_24.merge(df_23, on='timestamp', how='inner')

In [95]:
df_test.columns

Index(['timestamp', 'AMETHYSTS_curr', 'STARFRUIT_curr', 'BANANAS_past',
       'COCONUTS_past', 'PEARLS_past', 'PINA_COLADAS_past'],
      dtype='object')

In [44]:
def get_prev_returns(df, col, its):
    prev_col = f"{col}_prev_{its}_its"
    df[prev_col] = df[col].shift(its)
    df[f"{col}_returns_from_{its}_its_ago"] = (df[col] - df[prev_col]) / df[prev_col]
    df.drop(columns=[prev_col], inplace=True)
    return df

def get_future_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    df[f"{col}_returns_in_{its}_its"] = (df[future_col] - df[col]) / df[col]
    df.drop(columns=[future_col], inplace=True)
    return df

def get_centered_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    prev_col = f"{col}_prev_{its}_its"
    df[prev_col] = df[col].shift(its)
    df[f"{col}_returns_centered_with_{its}_its"] = (df[future_col] - df[prev_col])/df[prev_col]
    df.drop(columns=[prev_col], inplace=True)
    df.drop(columns=[future_col], inplace=True)
    return df

In [60]:
import os
import json
import io
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import scipy.stats as stats
from tqdm import tqdm

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
        sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
        sandbox_log = sections[0].strip()
        activities_log = sections[1].split('Trade History:')[0]
        trade_history = json.loads(sections[1].split('Trade History:')[1])
        market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
        trade_history_df = pd.json_normalize(trade_history)
        return market_data_df, trade_history_df

def get_future_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    df[f"{col}_returns_in_{its}_its"] = (df[future_col] - df[col]) / df[col]
    df.drop(columns=[future_col], inplace=True)
    return df

predictor_timeframes = [20]
responder_timeframes = [20]

results = []


for j in tqdm(range(-1,2)):
    for i in range(1, 4):
        df_24 = pd.read_csv(f'/users/eleves-a/2023/jules.frealle/Documents/IMC/2025_data_bottles1/prices_round_2_day_{j}.csv', sep=';')
        df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
        df_23 = pd.read_csv(f"/users/eleves-a/2023/jules.frealle/Documents/IMC/2023_data_bottles/prices_round_4_day_{i}.csv", sep=';')
        df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
        df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
        df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]
        
        df_test = df_24.merge(df_23, on='timestamp', how='inner')
        
        predictor_symbols = [col for col in df_test.columns if col.endswith('_past')]
        responder_symbols = [col for col in df_test.columns if col.endswith('_curr')]
        
        df_copy = df_test.copy()
        
        for responder_timeframe in responder_timeframes:
            for symbol in responder_symbols:
                df_copy = get_future_returns(df_copy, symbol, responder_timeframe)
            
            for predictor_timeframe in predictor_timeframes:
                for symbol in predictor_symbols:
                    df_copy = get_future_returns(df_copy, symbol, predictor_timeframe)
                
                for predictor_symbol in predictor_symbols:
                    for responder_symbol in responder_symbols:
                        feature_col = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its") and col.startswith(predictor_symbol)]
                        target_col = f"{responder_symbol}_returns_in_{responder_timeframe}_its"
                        
                        df_train = df_copy[feature_col + [target_col]].dropna()
                        
                        X = df_train[feature_col]
                        y = df_train[target_col]
                        
                        model = LinearRegression(fit_intercept=False)
                        model.fit(X, y)
                        
                        y_pred = model.predict(X)
                        
                        r2 = r2_score(y, y_pred)
                        _, p_value = stats.pearsonr(y, y_pred)
                        
                        results.append({
                            '2024_day': i,
                            '2023_day': j,
                            'predictor_symbol': predictor_symbol,
                            'responder_symbol': responder_symbol,
                            'r_squared': r2,
                            'p_value': p_value,
                            'equation': f"{target_col} = {model.coef_[0]:.4f} * {feature_col[0]}"
                        })
                
                future_cols_predictor = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its")]
                df_copy.drop(columns=future_cols_predictor, inplace=True)
            
            future_cols_responder = [col for col in df_copy.columns if col.endswith(f"_returns_in_{responder_timeframe}_its")]
            df_copy.drop(columns=future_cols_responder, inplace=True)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r_squared', ascending=False)

display(results_df.head(20))

100%|█████████████████████████████████████████████| 3/3 [00:04<00:00,  1.39s/it]


,2024_day,2023_day,predictor_symbol,responder_symbol,r_squared,p_value,equation
590,1,1,PEARLS_past,RAINFOREST_RESIN_curr,1.000000,0.000000e+00,RAINFOREST_RESIN_curr_returns_in_20_its = 1.00...
542,1,1,BANANAS_past,RAINFOREST_RESIN_curr,0.019023,1.157358e-43,RAINFOREST_RESIN_curr_returns_in_20_its = 0.06...
125,2,-1,DIP_past,PICNIC_BASKET2_curr,0.012126,1.545890e-29,PICNIC_BASKET2_curr_returns_in_20_its = -0.057...
73,1,-1,PINA_COLADAS_past,DJEMBES_curr,0.011620,8.299171e-28,DJEMBES_curr_returns_in_20_its = -0.0503 * PIN...
687,2,1,PICNIC_BASKET_past,SQUID_INK_curr,0.010505,7.711535e-25,SQUID_INK_curr_returns_in_20_its = -0.8147 * P...
113,2,-1,COCONUTS_past,DJEMBES_curr,0.010440,2.003220e-25,DJEMBES_curr_returns_in_20_its = -0.0655 * COC...
161,2,-1,PINA_COLADAS_past,DJEMBES_curr,0.009594,1.287452e-23,DJEMBES_curr_returns_in_20_its = -0.0455 * PIN...
7,1,-1,BAGUETTE_past,SQUID_INK_curr,0.009039,1.623939e-21,SQUID_INK_curr_returns_in_20_its = -0.3704 * B...
25,1,-1,COCONUTS_past,DJEMBES_curr,0.008646,3.285213e-21,DJEMBES_curr_returns_in_20_its = -0.0600 * COC...
587,1,1,PEARLS_past,KELP_curr,0.008420,9.971145e-22,KELP_curr_returns_in_20_its = 0.2033 * PEARLS_...


In [61]:
import os
import json
import io
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import scipy.stats as stats
from tqdm import tqdm

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
        sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
        sandbox_log = sections[0].strip()
        activities_log = sections[1].split('Trade History:')[0]
        trade_history = json.loads(sections[1].split('Trade History:')[1])
        market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
        trade_history_df = pd.json_normalize(trade_history)
        return market_data_df, trade_history_df


predictor_timeframes = [1]
responder_timeframes = [1]

results = []

for j in tqdm(range(-1,2)):
    for i in range(1, 4):
        df_24 = pd.read_csv(f'/users/eleves-a/2023/jules.frealle/Documents/IMC/2025_data_bottles1/prices_round_2_day_{j}.csv', sep=';')
        df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
        df_23 = pd.read_csv(f"/users/eleves-a/2023/jules.frealle/Documents/IMC/2023_data_bottles/prices_round_4_day_{i}.csv", sep=';')
        df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
        df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]
        
        df_test = df_24.merge(df_23, on='timestamp', how='inner')
        
        predictor_symbols = [col for col in df_test.columns if col.endswith('_past')]
        responder_symbols = [col for col in df_test.columns if col.endswith('_curr')]
        
        df_copy = df_test.copy()
        
        for responder_timeframe in responder_timeframes:
            for symbol in responder_symbols:
                df_copy = get_future_returns(df_copy, symbol, responder_timeframe)
            
            for predictor_timeframe in predictor_timeframes:
                for symbol in predictor_symbols:
                    df_copy = get_future_returns(df_copy, symbol, predictor_timeframe)
                
                for predictor_symbol in predictor_symbols:
                    for responder_symbol in responder_symbols:
                        feature_col = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its") and col.startswith(predictor_symbol)]
                        target_col = f"{responder_symbol}_returns_in_{responder_timeframe}_its"
                        
                        df_train = df_copy[feature_col + [target_col]].dropna()
                        
                        X = df_train[feature_col]
                        y = df_train[target_col]
                        
                        model = LinearRegression(fit_intercept=False)
                        model.fit(X, y)
                        
                        y_pred = model.predict(X)
                        
                        r2 = r2_score(y, y_pred)
                        _, p_value = stats.pearsonr(y, y_pred)
                        
                        results.append({
                            '2024_day': i,
                            '2023_day': j,
                            'predictor_symbol': predictor_symbol,
                            'responder_symbol': responder_symbol,
                            'r_squared': r2,
                            'p_value': p_value,
                            'equation': f"{target_col} = {model.coef_[0]:.4f} * {feature_col[0]}"
                        })
                
                future_cols_predictor = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its")]
                df_copy.drop(columns=future_cols_predictor, inplace=True)
            
            future_cols_responder = [col for col in df_copy.columns if col.endswith(f"_returns_in_{responder_timeframe}_its")]
            df_copy.drop(columns=future_cols_responder, inplace=True)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r_squared', ascending=False)

display(results_df.head(20))

100%|█████████████████████████████████████████████| 3/3 [00:04<00:00,  1.38s/it]


,2024_day,2023_day,predictor_symbol,responder_symbol,r_squared,p_value,equation
590,1,1,PEARLS_past,RAINFOREST_RESIN_curr,1.000000,0.000000e+00,RAINFOREST_RESIN_curr_returns_in_1_its = 1.000...
539,1,1,BANANAS_past,KELP_curr,0.035829,2.555028e-81,KELP_curr_returns_in_1_its = 0.2069 * BANANAS_...
542,1,1,BANANAS_past,RAINFOREST_RESIN_curr,0.033823,8.738571e-77,RAINFOREST_RESIN_curr_returns_in_1_its = 0.114...
587,1,1,PEARLS_past,KELP_curr,0.013855,3.427976e-32,KELP_curr_returns_in_1_its = 0.2077 * PEARLS_p...
187,3,-1,BANANAS_past,KELP_curr,0.007758,1.108508e-18,KELP_curr_returns_in_1_its = 0.1004 * BANANAS_...
543,1,1,BANANAS_past,SQUID_INK_curr,0.004326,4.591069e-11,SQUID_INK_curr_returns_in_1_its = 0.2384 * BAN...
597,1,1,PICNIC_BASKET_past,PICNIC_BASKET2_curr,0.002719,1.480520e-07,PICNIC_BASKET2_curr_returns_in_1_its = 0.0307 ...
191,3,-1,BANANAS_past,SQUID_INK_curr,0.002428,8.223810e-07,SQUID_INK_curr_returns_in_1_its = 0.1603 * BAN...
596,1,1,PICNIC_BASKET_past,PICNIC_BASKET1_curr,0.001808,2.107350e-05,PICNIC_BASKET1_curr_returns_in_1_its = 0.0211 ...
591,1,1,PEARLS_past,SQUID_INK_curr,0.001559,7.801190e-05,SQUID_INK_curr_returns_in_1_its = 0.2310 * PEA...


In [63]:
import os
import json
import io
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import scipy.stats as stats
from tqdm import tqdm

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
        sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
        sandbox_log = sections[0].strip()
        activities_log = sections[1].split('Trade History:')[0]
        trade_history = json.loads(sections[1].split('Trade History:')[1])
        market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
        trade_history_df = pd.json_normalize(trade_history)
        return market_data_df, trade_history_df

results = []

for j in tqdm(range(-1,2)):
    for i in range(1, 4):
        df_24 = pd.read_csv(f'/users/eleves-a/2023/jules.frealle/Documents/IMC/2025_data_bottles1/prices_round_2_day_{j}.csv', sep=';')
        df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
        df_23 = pd.read_csv(f"/users/eleves-a/2023/jules.frealle/Documents/IMC/2023_data_bottles/prices_round_4_day_{i}.csv", sep=';')
        df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
        df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
        df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]
        
        df_test = df_24.merge(df_23, on='timestamp', how='inner')
        
        predictor_symbols = [col for col in df_test.columns if col.endswith('_past')]
        responder_symbols = [col for col in df_test.columns if col.endswith('_curr')]
        
        for predictor_symbol in predictor_symbols:
            for responder_symbol in responder_symbols:
                X = df_test[[predictor_symbol]]
                y = df_test[responder_symbol]
                
                for fit_intercept in [True, False]:
                    model = LinearRegression(fit_intercept=fit_intercept)
                    model.fit(X, y)
                    
                    y_pred = model.predict(X)
                    
                    r2 = r2_score(y, y_pred)
                    _, p_value = stats.pearsonr(y, y_pred)
                    
                    if fit_intercept:
                        equation = f"{responder_symbol} = {model.intercept_:.4f} + {model.coef_[0]:.4f} * {predictor_symbol}"
                    else:
                        equation = f"{responder_symbol} = {model.coef_[0]:.4f} * {predictor_symbol}"
                    
                    results.append({
                        '2024_day': i,
                        '2023_day': j,
                        'predictor_symbol': predictor_symbol,
                        'responder_symbol': responder_symbol,
                        'fit_intercept': fit_intercept,
                        'r_squared': r2,
                        'p_value': p_value,
                        'equation': equation
                    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r_squared', ascending=False)

display(results_df.head(20))

100%|█████████████████████████████████████████████| 3/3 [00:05<00:00,  1.67s/it]


,2024_day,2023_day,predictor_symbol,responder_symbol,fit_intercept,r_squared,p_value,equation
1181,1,1,PEARLS_past,RAINFOREST_RESIN_curr,False,1.000000,0.0,RAINFOREST_RESIN_curr = 1.0000 * PEARLS_past
1180,1,1,PEARLS_past,RAINFOREST_RESIN_curr,True,1.000000,0.0,RAINFOREST_RESIN_curr = 0.0000 + 1.0000 * PEAR...
564,1,0,BERRIES_past,JAMS_curr,True,0.847275,0.0,JAMS_curr = 2158.3469 + 1.1403 * BERRIES_past
452,3,-1,DOLPHIN_SIGHTINGS_past,JAMS_curr,True,0.826881,0.0,JAMS_curr = 8564.7696 + -0.6089 * DOLPHIN_SIGH...
990,3,0,DOLPHIN_SIGHTINGS_past,SQUID_INK_curr,True,0.790457,0.0,SQUID_INK_curr = 5429.3069 + -1.1246 * DOLPHIN...
708,2,0,BAGUETTE_past,JAMS_curr,True,0.780900,0.0,JAMS_curr = 9957.3216 + -0.2735 * BAGUETTE_past
978,3,0,DOLPHIN_SIGHTINGS_past,DJEMBES_curr,True,0.760821,0.0,DJEMBES_curr = 16210.6547 + -0.8777 * DOLPHIN_...
900,3,0,BANANAS_past,JAMS_curr,True,0.752457,0.0,JAMS_curr = -541.6239 + 1.4954 * BANANAS_past
901,3,0,BANANAS_past,JAMS_curr,False,0.748127,0.0,JAMS_curr = 1.3819 * BANANAS_past
570,1,0,BERRIES_past,PICNIC_BASKET2_curr,True,0.743027,0.0,PICNIC_BASKET2_curr = 21046.6478 + 2.4062 * BE...


In [79]:
#df_24 = pd.read_csv(f'/users/eleves-a/2023/jules.frealle/Documents/IMC/2024_data_bottles/prices_round_4_day_{1}.csv', sep=';')
#df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

#df_23 = pd.read_csv(f"/users/eleves-a/2023/jules.frealle/Documents/IMC/2023_data_bottles/prices_round_2_day_{-1}.csv", sep=';')
#df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()


df_24 = pd.read_csv(f'/users/eleves-a/2023/jules.frealle/Documents/IMC/2025_data_bottles1/prices_round_2_day_1.csv', sep=';')
df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
df_23 = pd.read_csv(f"/users/eleves-a/2023/jules.frealle/Documents/IMC/2023_data_bottles/prices_round_4_day_1.csv", sep=';')
df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

In [84]:
df_23

product,timestamp,BAGUETTE,BANANAS,BERRIES,COCONUT_PREV,DIP,DIVING_GEAR,DOLPHIN_SIGHTINGS,PEARLS,PICNIC_BASKET,PINA_COLADAS,UKULELE
0,0,12000.0,4874.0,3919.5,8004.5,7000.0,98904.0,3047.0,10001.5,73365.0,15014.5,21000.0
1,100,12002.0,4876.0,3919.5,8004.0,7001.5,98899.5,3047.0,10003.0,73383.5,15012.5,21003.0
2,200,12002.5,4873.0,3919.5,8004.0,7001.5,98899.5,3048.0,10000.0,73385.0,15016.5,21008.5
3,300,12003.5,4874.0,3919.5,8005.0,7003.0,98895.0,3048.0,10000.0,73371.0,15015.5,21000.5
4,400,12004.0,4870.5,3920.5,8003.5,7002.0,98909.0,3048.0,10001.0,73342.5,15013.5,21000.5
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,999500,12255.0,4821.5,3850.0,7924.5,7093.0,99192.0,3075.0,10001.0,74310.5,14850.5,20966.0
9996,999600,12257.0,4821.5,3850.0,7925.0,7095.0,99185.5,3076.0,10000.0,74319.0,14854.0,20968.5
9997,999700,12258.5,4820.5,3850.5,7924.0,7096.5,99193.0,3076.0,9999.0,74329.5,14851.0,20966.0
9998,999800,12254.0,4824.0,3850.5,7924.5,7096.0,99187.5,3075.0,10000.0,74318.0,14850.5,20967.0


In [85]:
df = df_24[['timestamp', 'JAMS']].merge(df_23[['timestamp', 'BERRIES']], on='timestamp', how='inner')

In [86]:
df = get_future_returns(df, 'BERRIES', 50)
df = get_future_returns(df, 'JAMS', 50)

In [87]:
df_r2 = df.dropna()
r2_score(df_r2['BERRIES_returns_in_50_its'], df_r2['JAMS_returns_in_50_its'])

-0.4159317547888002

In [74]:
df['COCONUT_mult'] = 1 + df['COCONUT_PREV_returns_in_1_its']

KeyError: 'COCONUT_PREV_returns_in_1_its'

In [109]:
df['COCONUT_pred'] = df['COCONUT_CURR']*df['COCONUT_mult']

In [68]:
df["COCONUT_pred"] = df["COCONUT_pred"].round(0) + (df["COCONUT_pred"] % 1 >= 0.5) * 0.5

In [88]:
df['COCONUT_pred'] = df['COCONUT_pred'].shift(1)

In [75]:
df_r2 = df.dropna()
r2_score(df_r2['COCONUT_CURR'], df_r2['COCONUT_pred'])

KeyError: 'COCONUT_pred'

In [76]:
df.columns

Index(['timestamp', 'COCONUT_CURR', 'COCONUT_PREV',
       'COCONUT_PREV_returns_in_50_its', 'COCONUT_CURR_returns_in_50_its'],
      dtype='object', name='product')

In [77]:
import plotly.graph_objects as go

# Create traces for each return series
trace_curr_1 = go.Scatter(x=df['timestamp'], y=df['COCONUT_CURR_returns_in_1_its'],
                           mode='lines', name='Current Returns (1 iteration)')
trace_prev_1 = go.Scatter(x=df['timestamp'], y=df['COCONUT_PREV_returns_in_1_its'],
                           mode='lines', name='Previous Returns (1 iteration)')

trace_curr_20 = go.Scatter(x=df['timestamp'], y=df['COCONUT_CURR_returns_in_20_its'],
                            mode='lines', name='Current Returns (20 iterations)')
trace_prev_20 = go.Scatter(x=df['timestamp'], y=df['COCONUT_PREV_returns_in_20_its'],
                            mode='lines', name='Previous Returns (20 iterations)')

trace_curr_50 = go.Scatter(x=df['timestamp'], y=df['COCONUT_CURR_returns_in_50_its'],
                            mode='lines', name='Current Returns (50 iterations)')
trace_prev_50 = go.Scatter(x=df['timestamp'], y=df['COCONUT_PREV_returns_in_50_its'],
                            mode='lines', name='Previous Returns (50 iterations)')

# Create the figure and add the traces
fig = go.Figure()
fig.add_trace(trace_curr_1)
fig.add_trace(trace_prev_1)
fig.add_trace(trace_curr_20)
fig.add_trace(trace_prev_20)
fig.add_trace(trace_curr_50)
fig.add_trace(trace_prev_50)

# Set the layout
fig.update_layout(
    title='Returns Over Iterations',
    xaxis_title='Timestamp',
    yaxis_title='Returns',
    legend_title='Return Series'
)

# Display the plot
fig.show()

KeyError: 'COCONUT_CURR_returns_in_1_its'

In [70]:
# Initialize the first value of "COCONUT_stupid" as 8000
df.loc[0, 'COCONUT_stupid'] = 10000

# Calculate the subsequent values of "COCONUT_stupid" based on the previous row's values
for i in range(1, len(df)):
    df.loc[i, 'COCONUT_stupid'] = round(2*(df.loc[i-1, 'COCONUT_stupid'] * df.loc[i-1, 'COCONUT_mult']))/2
    

In [71]:
df

product,timestamp,COCONUT_CURR,COCONUT_PREV,COCONUT_PREV_returns_in_1_its,COCONUT_mult,COCONUT_pred,COCONUT_stupid
0,0,10000.0,8000.0,-0.000063,0.999938,9999.0,10000.0
1,100,9999.5,7999.5,-0.000125,0.999875,9998.0,9999.5
2,200,9998.5,7998.5,0.000125,1.000125,10000.5,9998.0
3,300,9999.5,7999.5,-0.000125,0.999875,9998.0,9999.0
4,400,9998.5,7998.5,-0.000188,0.999812,9997.5,9998.0
...,...,...,...,...,...,...,...
9995,999500,10135.5,8114.5,0.000062,1.000062,10136.0,10173.0
9996,999600,10136.5,8115.0,-0.000185,0.999815,10135.5,10173.5
9997,999700,10134.5,8113.5,0.000000,1.000000,10134.5,10171.5
9998,999800,10135.0,8113.5,0.000062,1.000062,10136.5,10171.5


In [73]:
df_r2 = df.copy()
df_r2 = df_r2.dropna()
r2_score(df_r2['COCONUT_CURR'], df_r2['COCONUT_stupid'])

0.7009595942688397

In [77]:
for i in tqdm(range(1, 4)):
    j = i - 2
    df_24 = pd.read_csv(f'2024_data_bottles/round-4-island-data-bottle/prices_round_4_day_{i}.csv', sep=';')
    df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

    df_23 = pd.read_csv(f"2023_data_bottles/island-data-bottle-round-2/prices_round_2_day_{j}.csv", sep=';')
    df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
    df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
    df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]

    df_test = df_24.merge(df_23, on='timestamp', how='inner')
    display(df_test[['timestamp','COCONUT_curr', 'COCONUTS_past']])

  0%|                                                                       | 0/3 [00:00<?, ?it/s]

,timestamp,COCONUT_curr,COCONUTS_past
0,0,10000.0,8000.0
1,100,9999.5,7999.5
2,200,9998.5,7998.5
3,300,9999.5,7999.5
4,400,9998.5,7998.5
...,...,...,...
9995,999500,10135.5,8114.5
9996,999600,10136.5,8115.0
9997,999700,10134.5,8113.5
9998,999800,10135.0,8113.5


,timestamp,COCONUT_curr,COCONUTS_past
0,0,10135.5,8114.5
1,100,10135.0,8114.0
2,200,10135.5,8114.5
3,300,10135.0,8114.0
4,400,10133.5,8113.0
...,...,...,...
9995,999500,9988.0,8002.0
9996,999600,9987.0,8001.0
9997,999700,9986.5,8001.0
9998,999800,9989.5,8003.0


 67%|██████████████████████████████████████████                     | 2/3 [00:00<00:00, 12.20it/s]

,timestamp,COCONUT_curr,COCONUTS_past
0,0,9991.0,8004.5
1,100,9990.0,8004.0
2,200,9990.5,8004.0
3,300,9991.5,8005.0
4,400,9990.0,8003.5
...,...,...,...
9995,999500,9883.5,7924.5
9996,999600,9884.5,7925.0
9997,999700,9883.5,7924.0
9998,999800,9884.0,7924.5


100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13.18it/s]


# price change space

In [37]:
def get_future_price_change(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    df[f"{col}_price_change_in_{its}_its"] = df[future_col] - df[col]
    df.drop(columns=[future_col], inplace=True)
    return df


predictor_timeframes = [1]
responder_timeframes = [1]

results = []

for i in tqdm(range(1, 4)):
    for j in range(-1, 2):
        df_24 = pd.read_csv(f'2024_data_bottles/round-4-island-data-bottle/prices_round_4_day_{i}.csv', sep=';')
        df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        
        df_23 = pd.read_csv(f"2023_data_bottles/island-data-bottle-round-2/prices_round_2_day_{j}.csv", sep=';')
        df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
        df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
        df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]
        
        df_test = df_24.merge(df_23, on='timestamp', how='inner')
        
        predictor_symbols = [col for col in df_test.columns if col.endswith('_past')]
        responder_symbols = [col for col in df_test.columns if col.endswith('_curr')]
        
        df_copy = df_test.copy()
        
        for responder_timeframe in responder_timeframes:
            for symbol in responder_symbols:
                df_copy = get_future_price_change(df_copy, symbol, responder_timeframe)
            
            for predictor_timeframe in predictor_timeframes:
                for symbol in predictor_symbols:
                    df_copy = get_future_price_change(df_copy, symbol, predictor_timeframe)
                
                for predictor_symbol in predictor_symbols:
                    for responder_symbol in responder_symbols:
                        feature_col = [col for col in df_copy.columns if col.endswith(f"_price_change_in_{predictor_timeframe}_its") and col.startswith(predictor_symbol)]
                        target_col = f"{responder_symbol}_price_change_in_{responder_timeframe}_its"
                        
                        df_train = df_copy[feature_col + [target_col]].dropna()
                        
                        X = df_train[feature_col]
                        y = df_train[target_col]
                        
                        model = LinearRegression(fit_intercept=False)
                        model.fit(X, y)
                        
                        y_pred = model.predict(X)
                        
                        r2 = r2_score(y, y_pred)
                        _, p_value = stats.pearsonr(y, y_pred)
                        
                        results.append({
                            '2024_day': i,
                            '2023_day': j,
                            'predictor_symbol': predictor_symbol,
                            'responder_symbol': responder_symbol,
                            'r_squared': r2,
                            'p_value': p_value,
                            'equation': f"{target_col} = {model.coef_[0]:.4f} * {feature_col[0]}"
                        })
                
                future_cols_predictor = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its")]
                df_copy.drop(columns=future_cols_predictor, inplace=True)
            
            future_cols_responder = [col for col in df_copy.columns if col.endswith(f"_returns_in_{responder_timeframe}_its")]
            df_copy.drop(columns=future_cols_responder, inplace=True)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r_squared', ascending=False)

print(results_df)

100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.08it/s]

    2024_day  2023_day   predictor_symbol responder_symbol  r_squared  \
34         2         0      COCONUTS_past     COCONUT_curr   0.763445   
66         3         1      COCONUTS_past     COCONUT_curr   0.755620   
2          1        -1      COCONUTS_past     COCONUT_curr   0.754282   
38         2         0  PINA_COLADAS_past     COCONUT_curr   0.471744   
70         3         1  PINA_COLADAS_past     COCONUT_curr   0.467022   
..       ...       ...                ...              ...        ...   
14         1         0  PINA_COLADAS_past     COCONUT_curr  -0.000169   
26         2        -1      COCONUTS_past     COCONUT_curr  -0.000181   
28         2        -1        PEARLS_past     COCONUT_curr  -0.000193   
42         2         1      COCONUTS_past     COCONUT_curr  -0.000196   
36         2         0        PEARLS_past     COCONUT_curr  -0.000199   

     p_value                                           equation  
34  0.000000  COCONUT_curr_price_change_in_1_its = 1.0036

In [38]:
display(results_df.head(20))

,2024_day,2023_day,predictor_symbol,responder_symbol,r_squared,p_value,equation
34,2,0,COCONUTS_past,COCONUT_curr,0.763445,0.000000e+00,COCONUT_curr_price_change_in_1_its = 1.0036 * ...
66,3,1,COCONUTS_past,COCONUT_curr,0.755620,0.000000e+00,COCONUT_curr_price_change_in_1_its = 0.9937 * ...
2,1,-1,COCONUTS_past,COCONUT_curr,0.754282,0.000000e+00,COCONUT_curr_price_change_in_1_its = 0.9975 * ...
38,2,0,PINA_COLADAS_past,COCONUT_curr,0.471744,0.000000e+00,COCONUT_curr_price_change_in_1_its = 0.3294 * ...
70,3,1,PINA_COLADAS_past,COCONUT_curr,0.467022,0.000000e+00,COCONUT_curr_price_change_in_1_its = 0.3237 * ...
6,1,-1,PINA_COLADAS_past,COCONUT_curr,0.462556,0.000000e+00,COCONUT_curr_price_change_in_1_its = 0.3287 * ...
35,2,0,COCONUTS_past,COCONUT_COUPON_curr,0.181570,0.000000e+00,COCONUT_COUPON_curr_price_change_in_1_its = 0....
3,1,-1,COCONUTS_past,COCONUT_COUPON_curr,0.170089,0.000000e+00,COCONUT_COUPON_curr_price_change_in_1_its = 0....
67,3,1,COCONUTS_past,COCONUT_COUPON_curr,0.142284,0.000000e+00,COCONUT_COUPON_curr_price_change_in_1_its = 0....
39,2,0,PINA_COLADAS_past,COCONUT_COUPON_curr,0.108424,1.745551e-251,COCONUT_COUPON_curr_price_change_in_1_its = 0....


In [59]:
df

product,timestamp,COCONUT_CURR,COCONUT_PREV,COCONUT_CURR_returns_in_1_its,COCONUT_mult,COCONUT_pred,COCONUT_stupid
0,0,10000.0,8000.0,-0.000050,0.999950,8000.5,8000.0
1,100,9999.5,7999.5,-0.000100,0.999900,7999.5,7999.5
2,200,9998.5,7998.5,0.000100,1.000100,7999.0,7998.5
3,300,9999.5,7999.5,-0.000100,0.999900,7999.5,7999.5
4,400,9998.5,7998.5,-0.000200,0.999800,7997.5,7998.5
...,...,...,...,...,...,...,...
9995,999500,10135.5,8114.5,0.000099,1.000099,8115.0,8095.0
9996,999600,10136.5,8115.0,-0.000197,0.999803,8113.0,8096.0
9997,999700,10134.5,8113.5,0.000049,1.000049,8114.5,8094.5
9998,999800,10135.0,8113.5,0.000000,1.000000,8114.5,8095.0


# roses

In [83]:
for i in tqdm(range(0, 3)):
    j = i
    df_24 = pd.read_csv(f'2024_data_bottles/round-3-island-data-bottle/prices_round_3_day_{i}.csv', sep=';')
    df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

    df_23 = pd.read_csv(f"2023_data_bottles/island-data-bottle-round-3/prices_round_3_day_{j}.csv", sep=';')
    df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
    df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
    df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]

    df_test = df_24.merge(df_23, on='timestamp', how='inner')
    print(df_test.columns)
    display(df_test[['timestamp','ROSES_curr', 'DIVING_GEAR_past']])

  0%|                                                                       | 0/3 [00:00<?, ?it/s]

Index(['timestamp', 'CHOCOLATE_curr', 'GIFT_BASKET_curr', 'ROSES_curr',
       'STRAWBERRIES_curr', 'BANANAS_past', 'BERRIES_past', 'COCONUTS_past',
       'DIVING_GEAR_past', 'DOLPHIN_SIGHTINGS_past', 'PEARLS_past',
       'PINA_COLADAS_past'],
      dtype='object')


,timestamp,ROSES_curr,DIVING_GEAR_past
0,0,15000.0,100000.0
1,100,15000.0,100000.0
2,200,15000.5,100002.5
3,300,15000.0,100000.0
4,400,14997.5,99994.5
...,...,...,...
9995,999500,14543.5,98899.0
9996,999600,14548.0,98909.5
9997,999700,14546.5,98906.0
9998,999800,14546.5,98907.5


 33%|█████████████████████                                          | 1/3 [00:00<00:00,  7.43it/s]

Index(['timestamp', 'CHOCOLATE_curr', 'GIFT_BASKET_curr', 'ROSES_curr',
       'STRAWBERRIES_curr', 'BANANAS_past', 'BERRIES_past', 'COCONUTS_past',
       'DIVING_GEAR_past', 'DOLPHIN_SIGHTINGS_past', 'PEARLS_past',
       'PINA_COLADAS_past'],
      dtype='object')


,timestamp,ROSES_curr,DIVING_GEAR_past
0,0,14545.5,98904.0
1,100,14543.5,98899.5
2,200,14543.5,98899.5
3,300,14541.5,98895.0
4,400,14547.5,98909.0
...,...,...,...
9995,999500,14421.0,99192.0
9996,999600,14418.0,99185.5
9997,999700,14421.5,99193.0
9998,999800,14419.0,99187.5


 67%|██████████████████████████████████████████                     | 2/3 [00:00<00:00,  8.59it/s]

Index(['timestamp', 'CHOCOLATE_curr', 'GIFT_BASKET_curr', 'ROSES_curr',
       'STRAWBERRIES_curr', 'BANANAS_past', 'BERRIES_past', 'COCONUTS_past',
       'DIVING_GEAR_past', 'DOLPHIN_SIGHTINGS_past', 'PEARLS_past',
       'PINA_COLADAS_past'],
      dtype='object')


,timestamp,ROSES_curr,DIVING_GEAR_past
0,0,14421.5,99192.5
1,100,14424.0,99199.0
2,200,14422.5,99195.5
3,300,14421.5,99194.0
4,400,14422.0,99195.0
...,...,...,...
9995,999500,14409.5,100174.5
9996,999600,14408.0,100171.0
9997,999700,14411.5,100179.5
9998,999800,14412.5,100182.0


100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.76it/s]


In [84]:
(98904.0-98915.0)/98915.0

-0.00011120659151796997

In [85]:
(14545.5-14550.5)/14550.5

-0.00034363080306518677

In [86]:
-0.00011120659151796997*3.07

-0.0003414042359601678

# ugh cross day over multiple augh 

In [131]:
import os
import json
import io
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import scipy.stats as stats
from tqdm import tqdm

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
        sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
        sandbox_log = sections[0].strip()
        activities_log = sections[1].split('Trade History:')[0]
        trade_history = json.loads(sections[1].split('Trade History:')[1])
        market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
        trade_history_df = pd.json_normalize(trade_history)
        return market_data_df, trade_history_df

def get_future_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    df[f"{col}_returns_in_{its}_its"] = (df[future_col] - df[col]) / df[col]
    df.drop(columns=[future_col], inplace=True)
    return df

predictor_timeframes = [20]
responder_timeframes = [20]

results = []

for i in tqdm(range(1, 5)):  # Iterate over rounds 1 to 5 for 2024 data
    if i == 2:
        continue
    for j in range(1, 5):  # Iterate over rounds 1 to 5 for 2023 data
        for k in range(i - 3, i):  # Iterate over CSV files indexed from round_number - 3 to round_number - 1
            df_24 = pd.read_csv(f'2024_data_bottles/round-{i}-island-data-bottle/prices_round_{i}_day_{k}.csv', sep=';')
            df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
            df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]
            for l in range(j - 3, j):  # Iterate over CSV files indexed from round_number - 3 to round_number - 1

                df_23 = pd.read_csv(f"2023_data_bottles/island-data-bottle-round-{j}/prices_round_{j}_day_{l}.csv", sep=';')
                df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()

                df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]

                df_test = df_24.merge(df_23, on='timestamp', how='inner')

                predictor_symbols = [col for col in df_test.columns if col.endswith('_past')]
                responder_symbols = [col for col in df_test.columns if col.endswith('_curr')]

                df_copy = df_test.copy()
        
        for responder_timeframe in responder_timeframes:
            for symbol in responder_symbols:
                df_copy = get_future_returns(df_copy, symbol, responder_timeframe)
            
            for predictor_timeframe in predictor_timeframes:
                for symbol in predictor_symbols:
                    df_copy = get_future_returns(df_copy, symbol, predictor_timeframe)
                
                for predictor_symbol in predictor_symbols:
                    for responder_symbol in responder_symbols:
                        feature_col = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its") and col.startswith(predictor_symbol)]
                        target_col = f"{responder_symbol}_returns_in_{responder_timeframe}_its"
                        
                        df_train = df_copy[feature_col + [target_col]].dropna()
                        
                        X = df_train[feature_col]
                        y = df_train[target_col]
                        
                        model = LinearRegression(fit_intercept=False)
                        model.fit(X, y)
                        
                        y_pred = model.predict(X)
                        
                        r2 = r2_score(y, y_pred)
                        _, p_value = stats.pearsonr(y, y_pred)
                        
                        results.append({
                            '2024_day': i,
                            '2023_day': j,
                            'predictor_symbol': predictor_symbol,
                            'responder_symbol': responder_symbol,
                            'r_squared': r2,
                            'p_value': p_value,
                            'equation': f"{target_col} = {model.coef_[0]:.4f} * {feature_col[0]}"
                        })
                
                future_cols_predictor = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its")]
                df_copy.drop(columns=future_cols_predictor, inplace=True)
            
            future_cols_responder = [col for col in df_copy.columns if col.endswith(f"_returns_in_{responder_timeframe}_its")]
            df_copy.drop(columns=future_cols_responder, inplace=True)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r_squared', ascending=False)

print(results_df)

100%|███████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.91s/it]

     2024_day  2023_day        predictor_symbol     responder_symbol  \
2           1         1             PEARLS_past       AMETHYSTS_curr   
150         4         2           COCONUTS_past         COCONUT_curr   
86          3         3        DIVING_GEAR_past           ROSES_curr   
154         4         2       PINA_COLADAS_past         COCONUT_curr   
151         4         2           COCONUTS_past  COCONUT_COUPON_curr   
..        ...       ...                     ...                  ...   
53          3         1             PEARLS_past     GIFT_BASKET_curr   
105         3         4            BANANAS_past     GIFT_BASKET_curr   
65          3         2             PEARLS_past     GIFT_BASKET_curr   
89          3         3  DOLPHIN_SIGHTINGS_past     GIFT_BASKET_curr   
57          3         2            BANANAS_past     GIFT_BASKET_curr   

     r_squared   p_value                                           equation  
2     1.000000  0.000000  AMETHYSTS_curr_returns_in_20_it

In [132]:
results_df.head(50)

,2024_day,2023_day,predictor_symbol,responder_symbol,r_squared,p_value,equation
2,1,1,PEARLS_past,AMETHYSTS_curr,1.000000,0.000000e+00,AMETHYSTS_curr_returns_in_20_its = 1.0000 * PE...
150,4,2,COCONUTS_past,COCONUT_curr,0.985482,0.000000e+00,COCONUT_curr_returns_in_20_its = 0.9894 * COCO...
86,3,3,DIVING_GEAR_past,ROSES_curr,0.945804,0.000000e+00,ROSES_curr_returns_in_20_its = 2.8826 * DIVING...
154,4,2,PINA_COLADAS_past,COCONUT_curr,0.543138,0.000000e+00,COCONUT_curr_returns_in_20_its = 0.5347 * PINA...
151,4,2,COCONUTS_past,COCONUT_COUPON_curr,0.167053,0.000000e+00,COCONUT_COUPON_curr_returns_in_20_its = 7.4531...
85,3,3,DIVING_GEAR_past,GIFT_BASKET_curr,0.113514,5.777558e-282,GIFT_BASKET_curr_returns_in_20_its = 0.5983 * ...
155,4,2,PINA_COLADAS_past,COCONUT_COUPON_curr,0.082240,3.097850e-188,COCONUT_COUPON_curr_returns_in_20_its = 3.8047...
1,1,1,BANANAS_past,STARFRUIT_curr,0.019391,9.342669e-45,STARFRUIT_curr_returns_in_20_its = 0.1414 * BA...
0,1,1,BANANAS_past,AMETHYSTS_curr,0.017718,1.066069e-40,AMETHYSTS_curr_returns_in_20_its = 0.0586 * BA...
78,3,3,BERRIES_past,ROSES_curr,0.007002,5.545065e-17,ROSES_curr_returns_in_20_its = -0.1139 * BERRI...


# roses basket

In [146]:
def get_future_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    df[f"{col}_returns_in_{its}_its"] = (df[future_col] - df[col]) / df[col]
    df.drop(columns=[future_col], inplace=True)
    return df

predictor_timeframes = [50]
responder_timeframes = [50]

results = []

for i in tqdm(range(0, 3)):
    df_24 = pd.read_csv(f'2024_data_bottles/round-3-island-data-bottle/prices_round_3_day_{i}.csv', sep=';')
    df_24 = df_24.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
    
    df_23 = pd.read_csv(f"2023_data_bottles/island-data-bottle-round-3/prices_round_3_day_{i}.csv", sep=';')
    df_23 = df_23.pivot(columns='product', values='mid_price', index='timestamp').reset_index()
    
    df_23.columns = [df_23.columns[0]] + [col + '_past' for col in df_23.columns[1:]]
    df_24.columns = [df_24.columns[0]] + [col + '_curr' for col in df_24.columns[1:]]
    
    df_test = df_24.merge(df_23, on='timestamp', how='inner')
    
    predictor_symbol = 'DIVING_GEAR_past'
    responder_symbol = 'GIFT_BASKET_curr'
    
    df_copy = df_test.copy()
    df_copy['ROSES_IMPLIED_curr'] = df_copy['GIFT_BASKET_curr'] - 4*df_copy['CHOCOLATE_curr'] - 6*df_copy['STRAWBERRIES_curr']
     
    for responder_timeframe in responder_timeframes:
        df_copy = get_future_returns(df_copy, responder_symbol, responder_timeframe)
        
        for predictor_timeframe in predictor_timeframes:
            df_copy = get_future_returns(df_copy, predictor_symbol, predictor_timeframe)
            
            feature_col = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its") and col.startswith(predictor_symbol)]
            target_col = f"{responder_symbol}_returns_in_{responder_timeframe}_its"
            
            df_train = df_copy[feature_col + [target_col]].dropna()
            
            X = df_train[feature_col]
            y = df_train[target_col]
            
            model = LinearRegression(fit_intercept=False)
            model.fit(X, y)
            
            y_pred = model.predict(X)
            
            r2 = r2_score(y, y_pred)
            _, p_value = stats.pearsonr(y, y_pred)
            
            results.append({
                'day': i,
                'predictor_symbol': predictor_symbol,
                'responder_symbol': responder_symbol,
                'r_squared': r2,
                'p_value': p_value,
                'equation': f"{target_col} = {model.coef_[0]:.4f} * {feature_col[0]}"
            })
            
            future_cols_predictor = [col for col in df_copy.columns if col.endswith(f"_returns_in_{predictor_timeframe}_its")]
            df_copy.drop(columns=future_cols_predictor, inplace=True)
        
        future_cols_responder = [col for col in df_copy.columns if col.endswith(f"_returns_in_{responder_timeframe}_its")]
        df_copy.drop(columns=future_cols_responder, inplace=True)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r_squared', ascending=False)

print(results_df)

100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.98it/s]

   day  predictor_symbol  responder_symbol  r_squared        p_value  \
0    0  DIVING_GEAR_past  GIFT_BASKET_curr   0.177645   0.000000e+00   
1    1  DIVING_GEAR_past  GIFT_BASKET_curr   0.137903   0.000000e+00   
2    2  DIVING_GEAR_past  GIFT_BASKET_curr   0.094521  3.612523e-262   

                                            equation  
0  GIFT_BASKET_curr_returns_in_50_its = 0.6923 * ...  
1  GIFT_BASKET_curr_returns_in_50_its = 0.6587 * ...  
2  GIFT_BASKET_curr_returns_in_50_its = 0.5494 * ...  


In [147]:
results_df

,day,predictor_symbol,responder_symbol,r_squared,p_value,equation
0,0,DIVING_GEAR_past,GIFT_BASKET_curr,0.177645,0.000000e+00,GIFT_BASKET_curr_returns_in_50_its = 0.6923 * ...
1,1,DIVING_GEAR_past,GIFT_BASKET_curr,0.137903,0.000000e+00,GIFT_BASKET_curr_returns_in_50_its = 0.6587 * ...
2,2,DIVING_GEAR_past,GIFT_BASKET_curr,0.094521,3.612523e-262,GIFT_BASKET_curr_returns_in_50_its = 0.5494 * ...
